In [1]:
import constant as c
import functions as f
import pandas as pd
import os
from os.path import join
from itertools import combinations
import numpy as np
from scipy.spatial.distance import euclidean, hamming
import collections

from numpy import matlib

import proxmin
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
from imblearn.metrics import geometric_mean_score
from sklearn.preprocessing import StandardScaler

from joblib import dump, load



%matplotlib inline


In [2]:
ant_dataframe = []

for filename in os.listdir(c.ant):
    tmp = pd.read_csv(join(c.ant, filename))
    ant_dataframe.append(tmp)

ant_dataframe = pd.concat(ant_dataframe)

In [3]:
needed = ['name', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features_withbug = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc', 'bug']
features = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom',
       'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic',
       'cbm', 'amc', 'max_cc', 'avg_cc']


distances = ['e', 'm' , 'c']

In [4]:
versions = ant_dataframe.version.unique()
names = ant_dataframe.name.unique()
version_tuple = [(x, y) for x, y in zip(versions[0::1],versions[1::1])]

In [5]:
version_tuple[2:]

[(1.5, 1.6), (1.6, 1.7)]

In [6]:
distances[-1]

'c'

In [9]:
for v in version_tuple:
    print("__________________________")
    previous = ant_dataframe.loc[ant_dataframe.version == v[0]]
    current = ant_dataframe.loc[ant_dataframe.version == v[1]]
    previous = previous.sort_values('name').reset_index()
    current = current.sort_values('name').reset_index()
    previous = previous.drop(['index'], axis = 1)
    current = current.drop(['index'], axis = 1)
    modules_previous = previous.name.unique()
    modules_current = current.name.unique()
    for k in distances[-1]:
        print("PREVIOUS:", previous.version[0])
        print("CURRENT:", current.version[0])
        print("DISTANCE:", k)
        D = f.create_D(current, previous, features, k)
        idx = f.runDS3(D, reg = .0e-10, verbose = False)
        print('______WITH DS3_____')
        f.run_logisticRegression(previous, current, idx, ds3 = True, verbose = False, plot = False )
        #print('______WITHOUT DS3______')
        #f.run_logisticRegression(previous, current, None, ds3 = False, verbose = False, plot = False )
        #break
    
    print("__________________________")
    #break



    

__________________________
PREVIOUS: 1.3
CURRENT: 1.4
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.7415730337078652
F-Measure :  0.256364974245769
G-Mean : 0.44456605437946256
Balance : [0.36154444 0.37300308 0.29288167 0.2927905 ]
__________________________
__________________________
PREVIOUS: 1.4
CURRENT: 1.5
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.8668941979522184
F-Measure :  0.411415326669564
G-Mean : 0.544890667941091
Balance : [0.46811517 0.48638339 0.29289322]
__________________________
__________________________
PREVIOUS: 1.5
CURRENT: 1.6
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.7464387464387464
F-Measure :  0.1167875700982662
G-Mean : 0.3354555048369476
Balance : [0.43118123 0.38426847 0.29288995 0.29289322 0.29289322 0.29289322
 0.29289322 0.29289322 0.29289322]
__________________________
__________________________
PREVIOUS: 1.6
CURRENT: 1.7
DISTANCE: c


E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in longlong_scalars
  for (a, b) in zip(A, B)])
E:\GitHub\progetto-tesi\functions.py:29: RuntimeWarning: invalid value encountered in double_scalars
  for (a, b) in zip(A, B)])


______WITH DS3_____
- LogisticRegression score: 0.7516778523489933
F-Measure :  0.13668950875843427
G-Mean : 0.3583146385126642
Balance : [0.48777604 0.31502967 0.36323209 0.33702992 0.29289322 0.29289257
 0.46966219 0.29289322 0.29289322 0.29289066]
__________________________


In [10]:
previous.iloc[idx]

,project,version,name,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.6,org.apache.tools.ant.AntClassLoader,49,2,1,24,126,926,18,...,0.700000,1,0.441558,0.163462,1,5,29.448980,12,1.9796,10
1,ant,1.6,org.apache.tools.ant.AntTypeDefinition,20,1,2,11,45,112,7,...,0.857143,0,0.000000,0.316667,0,0,25.800000,11,3.0000,0
2,ant,1.6,org.apache.tools.ant.BuildEvent,11,2,0,12,15,13,9,...,1.000000,3,0.200000,0.220779,0,0,7.272727,1,0.7273,0
3,ant,1.6,org.apache.tools.ant.BuildException,15,4,0,243,27,0,242,...,1.000000,1,0.619048,0.322222,1,3,9.333333,2,0.6667,0
5,ant,1.6,org.apache.tools.ant.BuildLogger,4,1,0,5,4,6,4,...,0.000000,0,0.000000,0.500000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,ant,1.6,org.apache.tools.ant.util.regexp.RegexpMatcher,6,1,0,11,6,15,10,...,0.000000,0,0.000000,0.722222,0,0,0.000000,1,1.0000,0
331,ant,1.6,org.apache.tools.mail.ErrorInQuitException,1,4,0,1,3,0,1,...,0.000000,0,1.000000,1.000000,0,0,5.000000,0,0.0000,0
336,ant,1.6,org.apache.tools.tar.TarConstants,0,1,0,1,0,0,1,...,0.000000,0,0.000000,0.000000,0,0,0.000000,0,0.0000,1
341,ant,1.6,org.apache.tools.zip.AsiExtraField,20,1,0,4,42,130,0,...,1.000000,1,0.000000,0.284211,0,0,17.450000,3,1.1500,0
